# Ingest another dataset

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
import readfcs

lb.settings.species = "human"

In [ ]:
ln.track()

Let us validate and register another `.fcs` file:

## Access ![](https://img.shields.io/badge/Access-10b981)

In [ ]:
filepath = ln.dev.datasets.file_fcs()

adata = readfcs.read(filepath)

This `AnnData` object does not require filtering, normalizing or formatting, hence, there is no ![](https://img.shields.io/badge/Transform-10b981) step.

## Validate ![](https://img.shields.io/badge/Validate-10b981) 

First, let's standardize the cell markers and validate them:

In [ ]:
adata.var.index = lb.CellMarker.standardize(adata.var.index)
validated = lb.CellMarker.validate(adata.var.index)

Next, register non-validated markers from Bionty:

In [ ]:
records = lb.CellMarker.from_values(adata.var.index[~validated])
ln.save(records)

Now they pass validation except for non-markers: `'FSC-A'`, `'FSC-H'`

In [ ]:
lb.CellMarker.validate(adata.var.index);

## Register ![](https://img.shields.io/badge/Register-10b981) 

In [ ]:
modalities = ln.Modality.lookup()
features = ln.Feature.lookup()
efs = lb.ExperimentalFactor.lookup()
species = lb.Species.lookup()

In [ ]:
file = ln.File.from_anndata(
    adata,
    description="My fcs file",
    field=lb.CellMarker.name,
    modality=modalities.protein,
)

In [ ]:
file.save()

In [ ]:
file.labels.add(efs.fluorescence_activated_cell_sorting, features.assay)
file.labels.add(species.human, features.species)

In [ ]:
file.features

View data flow:

In [ ]:
file.view_flow()